## 實戰演練三

請根據demo3.ipynb 內的描述，將深度學習模型匯出，並透過建立一 API 讓模型上線，使用者可以調用該API 輸入資料並得到預測結果


### 問題一：使用Keras 建立深度學習模型預測客戶是否流失 (請建立一個 11 ->5 -> 1 的ANN)

In [10]:
#@markdown 資料讀取與前處理
import pandas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 讀取資料
df = pandas.read_csv('https://raw.githubusercontent.com/ywchiu/tibamedl/master/Data/Churn_Modelling.csv', header = 0 )

# 資料前處理
df = df.iloc[:,3:]
geography = pandas.get_dummies(df['Geography'])
gender = pandas.get_dummies(df['Gender'])

del geography['France']
del gender['Male']

df = pandas.concat([geography, gender, df], axis = 1)
del df['Gender']
del df['Geography']
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
sc = StandardScaler()
X = sc.fit_transform(X)
print('X data\n============\n', X[0:3,:])
print('\ny data\n============\n', y[0:3])
print('\nX\n============\n',X.shape)
print('\ny\n============\n',y.shape)

X data
 [[-0.57873591 -0.57380915  1.09598752 -0.32622142  0.29351742 -1.04175968
  -1.22584767 -0.91158349  0.64609167  0.97024255  0.02188649]
 [-0.57873591  1.74273971  1.09598752 -0.44003595  0.19816383 -1.38753759
   0.11735002 -0.91158349 -1.54776799  0.97024255  0.21653375]
 [-0.57873591 -0.57380915  1.09598752 -1.53679418  0.29351742  1.03290776
   1.33305335  2.52705662  0.64609167 -1.03067011  0.2406869 ]]

y data
 0    1
1    0
2    1
Name: Exited, dtype: int64

X
 (10000, 11)

y
 (10000,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
# 請完成以下程式碼

model.add(Dense(1, activation='sigmoid'))

In [13]:
#@markdown 檢視答案
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(5,activation='relu', input_shape= (11,)))
model.add(Dense(1, activation='sigmoid'))


In [14]:
model.compile(optimizer='adam', 
       loss = 'binary_crossentropy', 
       metrics = ['accuracy'])

In [ ]:
history = model.fit(
    x = X, 
    y = y,
    epochs = 10,
    verbose = 1)

### 問題二：請匯出深度學習模型至目錄夾sample_model

In [ ]:
model

In [ ]:
#@markdown 檢視答案
model.save('mymodel')
! mkdir -p /content/predict_model/1/
! mv /content/mymodel/* /content/predict_model/1/



### 問題三：請使用 Tensorflow Serving 佈署模型

In [ ]:
#@markdown 安裝Tensorflow Serving 
! echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -
! apt-get update && apt-get install tensorflow-model-server  /dev/null

In [ ]:
tensorflow_model_server 

In [ ]:
#@markdown 檢視答案
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=saved_model \
  --model_base_path="/content/predict_model/" >server.log 2>&1